In [ ]:
from sage.misc.prandom import randint
from sage.rings.all import ZZ, IntegerModRing, RR, PolynomialRing, GF
from sage.arith.all import next_prime, euler_phi, crt, xgcd
from sage.functions.log import log
from sage.functions.other import ceil
from sage.misc.functional import cyclotomic_polynomial, round
import numpy as np
from random import randint

# Parameter Generation

In [ ]:
d = 3
m = 3
N = 2**12
n = euler_phi(N)
log_q = 20 # weird sagemath behavior when this gets too big ca 50
evaluation_degree = d*m
D = d*m
tiny_primes = [2,3,5,7,11,13,17,19,23,29]
tiny_prime_product = 1
for prime in tiny_primes:
    tiny_prime_product *= prime

In [ ]:
var('Z')
g = cyclotomic_polynomial(N, var='Z')

In [ ]:
primes = []
q = 1
i = 1
while log(q,2).n()<log_q:
    p = N*i+1
    if p.is_prime():
        q *= p
        primes.append(p)
    i += 1
primes

In [ ]:
log(q,2).n()

In [ ]:
Z_q = IntegerModRing(q)
Q = Z_q['Z'].quo(g,Z)

In [ ]:
var('Y')
h = Q['Y'](1)
monomials = []
assert D < q
for i in range(D):
    mon = (Q[Y](Y) - Q[Y](i))
    h *= mon
    monomials.append(mon)
R = Q['Y'].quo(h,Y)

# BV11

In [ ]:
s = Q.random_element()

In [ ]:
def encrypt(s,mu):
    assert mu in Z_q
    assert s in Q
    a = R(Q.random_element())
    
    # TODO make the noise real
    e = randint(0,2)

    b = a*R(s)+d*e+mu
    return -a*R(Y)+b

In [ ]:
c1 = encrypt(s,1)
c2 = encrypt(s,2)
c = c1 * c2

In [ ]:
def decrypt(s,c):
    a = Q(c.lift()(Y=s)).lift()(Z=0).lift()
    return mod(a,d)

In [ ]:
decrypt(s,c)

# Database Interpolation

In [ ]:
P = PolynomialRing(GF(d),m,'x')
vars = P.gens()
vars

In [ ]:
a=np.array([[[1,1,0],[2,0,1],[2,0,1]],[[1,1,1],[0,0,2],[2,0,1]],[[1,1,1],[0,0,2],[2,0,1]]],dtype=np.int64)
b=[1,5,3]
def multivariate_interpolation(a, vars, m):
    # assertion: a is an m dimensional cube with length d
    if m==1:
        return GF(d)[vars[0]].lagrange_polynomial([(i,a[i])for i in range(d)])
    else:
        cs = np.zeros(tuple([d]*m), dtype=np.int64)
        for x in GF(d)**(m-1):
            g_x = GF(d)[vars[-1]].lagrange_polynomial([(i,a[tuple(x)][i])for i in range(d)])
            c_x = g_x.list()
            for i in range(len(c_x)):
                cs[tuple(x)][i] = c_x[i]

        f_is = []
        f = 0
        for i in GF(d):
            f_i = multivariate_interpolation(cs[..., i], vars[:-1], m-1)
            f_is.append(f_i)
            f += f_i * vars[-1]**i
        return f
f = multivariate_interpolation(a,vars,3)
print(f)
f(0,1,0)

# Eval

In [ ]:
def normal_eval(Q,f,cs):
    f_prime =f.change_ring(Q)
    return f_prime(cs)

In [ ]:
normal_eval(R,f,[c1,c2,c1])

In [ ]:
CRT = []
for i in range(len(monomials)):
    a = [1 if i == j else 0 for j in range(len(monomials))]
    CRT.append(Q[Y](Z_q[Y,Z](crt(a,[Z_q[Y](Z_q[Y,Z](mon)) for mon in monomials]))))

In [ ]:
def split_R(c):
    return [Q[Y].quo(mon)(c.lift()) for mon in monomials]

def combine_R(cs):
    result = Q[Y](0)
    for i in range(len(cs)):
        result += cs[i].lift()*CRT[i]
    return R(result)

In [ ]:
def split_Q_q(c):
    c = Z_q[Z](Z_q[Y,Z](c))
    return [IntegerModRing(prime)[Z](c) for prime in primes]

def combine_Q_q(cs):
    return Z_q[Z](crt([IntegerRing()[Z](c) for c in cs],primes))

In [ ]:
def split_Q_p(c,p):
    Z_p = IntegerModRing(p)
    w_n = Z_p.multiplicative_generator()**((p-1)//N)
    assert w_n.multiplicative_order() == N
    frac_ps = [Z_p[Z](Z)-Z_p[Z](w_n**i) for i in IntegerModRing(N).list_of_elements_of_multiplicative_group()]
    return [Z_p[Z].quo(frac_p,Z)(c) for frac_p in frac_ps]

def combine_Q_p(cs,p):
    Z_p = IntegerModRing(p)
    w_n = Z_p.multiplicative_generator()**((p-1)//N)
    assert w_n.multiplicative_order() == N
    frac_ps = [Z_p[Z](Z)-Z_p[Z](w_n**i) for i in IntegerModRing(N).list_of_elements_of_multiplicative_group()]
    return Q(crt([c.lift() for c in cs],frac_ps))


In [ ]:
def split_Z(c,M):
    while tiny_prime_product < M:
        tiny_primes.append(next_prime(tiny_primes[-1]))
        tiny_prime_product *= tiny_primes[-1]
    used_primes_product = 1
    i = 0
    while used_primes_product < M:
        used_primes_product *= tiny_primes[i]
        i += 1
    return [IntegerModRing(prime)(c) for prime in tiny_primes[:i]]

def combine_Z(cs,M):
    while tiny_prime_product < M:
        tiny_primes.append(next_prime(tiny_primes[-1]))
        tiny_prime_product *= tiny_primes[-1]
    used_primes_product = 1
    i = 0
    while used_primes_product < M:
        used_primes_product *= tiny_primes[i]
        i += 1
    return crt([IntegerRing()(c) for c in cs],tiny_primes[:i])

In [ ]:
(primes[-1],m)